In [1]:
%load_ext autoreload
%autoreload 2
import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
import hist

import sys
sys.path.append("../..")
from analyzer.dumperReader.reader import *
from analyzer.dumperReader.tracksters import trackster_joinSupercluster, tracksters_groupBy, _convertTsToDataframe

In [7]:
reader = DumperReader("/data_cms_upgrade/cuisset/supercls/alessandro_electrons/reco-v7/ticlDumper_201.root") #"../runs_results/histo.root:ticlDumper")

In [8]:
reader.tracksters_zipped

<Array [[{ts_id: 0, time: ..., ...}, ...], ...] type='100 * var * trackster...'>

In [10]:
reader.supercluster_df

ts_id
eventInternal supercls_id ts_in_supercls_id       
0             0           0                   1735
                          1                   1509
              1           0                    863
                          1                    284
              2           0                    293
...                                            ...
99            2333        0                   2338
              2334        0                   2339
              2335        0                   2340
              2336        0                   2341
              2337        0                   2342

[265022 rows x 1 columns]

In [ ]:
pd.DataFrame([{}])

In [36]:
getCPToSuperclusterProperties(reader.supercluster_all_df, reader.assocs_bestScore_simToReco_df, 
        reader.tracksters_zipped[["ts_id", "raw_energy", "raw_em_energy", "regressed_energy"]], reader.simTrackstersCP_df)

supercls_id  raw_energy_seed  \
eventInternal caloparticle_id                                 
0             1                          1       263.553101   
              0                          4       272.781616   
1             1                          0        98.968246   
              0                        560       100.469704   
2             0                          1       405.580688   
...                                    ...              ...   
97            0                          2       122.704117   
98            1                          2       276.707581   
              0                          3       291.213531   
99            1                          2       450.246552   
              0                        562       462.556213   

                               raw_em_energy_seed  regressed_energy_seed  \
eventInternal caloparticle_id                                              
0             1                        263.553101             279.278961   
              0                        272.781616             290.114594   
1             1                         98.968246             104.658203   
              0                        100.469704             109.036896   
2             0                        404.826080             426.025208   
...                                           ...                    ...   
97            0                        122.704117             131.418762   
98            1                        276.449371             293.732361   
              0                        290.940674             307.935364   
99            1                        449.764038             472.368683   
              0                        462.556213             484.305634   

                               raw_energy_supercls_sum  \
eventInternal caloparticle_id                            
0             1                             278.767120   
              0                             276.298523   
1             1                             111.944420   
              0                             100.469704   
2             0                             422.815399   
...                                                ...   
97            0                             128.007904   
98            1                             282.845428   
              0                             294.921021   
99            1                             455.960449   
              0                             462.556213   

                               raw_em_energy_supercls_sum  \
eventInternal caloparticle_id                               
0             1                                278.767120   
              0                                276.298523   
1             1                                111.944420   
              0                                100.469704   
2             0                                419.481598   
...                                                   ...   
97            0                                128.007904   
98            1                                282.587219   
              0                                294.648163   
99            1                                455.477936   
              0                                462.556213   

                               regressed_energy_supercls_sum  \
eventInternal caloparticle_id                                  
0             1                                   296.012756   
              0                                   296.585327   
1             1                                   117.887543   
              0                                   109.036896   
2             0                                   470.273895   
...                                                      ...   
97            0                                   139.016571   
98            1                                   308.233307   
              0                  

In [34]:
supercluster_all_df = reader.supercluster_all_df
assocs_bestScore_simToReco_df = reader.assocs_bestScore_simToReco_df
tracksters_all = reader.tracksters_zipped[["ts_id", "raw_energy", "raw_em_energy", "regressed_energy"]]
simTrackstersCP_df = reader.simTrackstersCP_df

In [35]:
tracksters_all = _convertTsToDataframe(tracksters_all)
# swap index around so we have Index : eventInternal, ts_id
df_mainAssoc = assocs_bestScore_simToReco_df.reset_index("caloparticle_id").set_index("ts_id", append=True)
df_mainAssoc

caloparticle_id     score     sharedE
eventInternal ts_id                                       
0             597                  0  0.000826  271.014709
              1784                 1  0.004819  262.696564
1             555                  0  0.006235   97.221283
              1901                 1  0.028445   92.765564
2             609                  0  0.003404  391.971100
...                              ...       ...         ...
97            1697                 1  0.088977   95.387070
98            614                  0  0.003265  281.200684
              2265                 1  0.011668  267.985260
99            557                  0  0.013765  401.091278
              2235                 1  0.012369  405.639374

[200 rows x 3 columns]

In [29]:
df_mainAssoc = df_mainAssoc.join(_convertTsToDataframe(tracksters_all).rename(columns={str(col) : str(col)+"_seed" for col in tracksters_all.columns}))
df_mainAssoc

caloparticle_id     score     sharedE  raw_energy_seed  \
eventInternal ts_id                                                           
0             597                  0  0.000826  271.014709       272.781616   
              1784                 1  0.004819  262.696564       263.553101   
1             555                  0  0.006235   97.221283       100.469704   
              1901                 1  0.028445   92.765564        98.968246   
2             609                  0  0.003404  391.971100       405.580688   
...                              ...       ...         ...              ...   
97            1697                 1  0.088977   95.387070        96.511948   
98            614                  0  0.003265  281.200684       291.213531   
              2265                 1  0.011668  267.985260       276.707581   
99            557                  0  0.013765  401.091278       462.556213   
              2235                 1  0.012369  405.639374       450.246552   

                     raw_em_energy_seed  regressed_energy_seed  
eventInternal ts_id                                             
0             597            272.781616             290.114594  
              1784           263.553101             279.278961  
1             555            100.469704             109.036896  
              1901            98.968246             104.658203  
2             609            404.826080             426.025208  
...                                 ...                    ...  
97            1697            96.511948             104.034645  
98            614            290.940674             307.935364  
              2265           276.449371             293.732361  
99            557            462.556213             484.305634  
              2235           449.764038             472.368683  

[200 rows x 6 columns]

In [31]:

# Join best associated trackster for each CP to all other tracksters in same supercluster
# keep caloparticle_id for later
tsInSupercls_df = trackster_joinSupercluster(df_mainAssoc, tracksters_all, supercluster_all_df, tracksterColumnsToKeep=["caloparticle_id", "raw_energy_seed"])
tsInSupercls_df

caloparticle_id  raw_energy_seed  supercls_id  \
eventInternal ts_id                                                  
0             597                  0       272.781616            4   
              157                  0       272.781616            4   
              905                  0       272.781616            4   
              1784                 1       263.553101            1   
              1785                 1       263.553101            1   
...                              ...              ...          ...   
99            557                  0       462.556213          562   
              2235                 1       450.246552            2   
              2262                 1       450.246552            2   
              1679                 1       450.246552            2   
              2351                 1       450.246552            2   

                     ts_in_supercls_id  raw_energy  raw_em_energy  \
eventInternal ts_id                                                 
0             597                    0  272.781616     272.781616   
              157                    0    2.697093       2.697093   
              905                    0    0.819827       0.819827   
              1784                   0  263.553101     263.553101   
              1785                   0   15.214010      15.214010   
...                                ...         ...            ...   
99            557                    0  462.556213     462.556213   
              2235                   0  450.246552     449.764038   
              2262                   0    1.695445       1.695445   
              1679                   0    2.164714       2.164714   
              2351                   0    1.853734       1.853734   

                     regressed_energy  
eventInternal ts_id                    
0             597          290.114594  
              157            6.470734  
              905            0.000000  
              1784         279.278961  
              1785          16.733791  
...                               ...  
99            557          484.305634  
              2235         472.368683  
              2262           4.597928  
              1679           5.815349  
              2351           4.656355  

[683 rows x 7 columns]

In [32]:

# aggregate tracksters in superclusters
supercls_grouped_df = (tsInSupercls_df
    .groupby(["eventInternal", "supercls_id"])
    .agg(**tracksters_groupBy(tsInSupercls_df, suffix="_supercls_sum"), caloparticle_id=pd.NamedAgg("caloparticle_id", "first"))
    .reset_index("supercls_id")
    .set_index("caloparticle_id", append=True)
)

# merge with CaloParticle information
merged_CP = pd.merge(supercls_grouped_df, simTrackstersCP_df,
                how="left", right_index=True, left_index=True)
merged_CP.rename(columns={str(col) : str(col)+"_CP" for col in simTrackstersCP_df.columns})

supercls_id  raw_energy_supercls_sum  \
eventInternal caloparticle_id                                         
0             1                          1               278.767120   
              0                          4               276.298523   
1             1                          0               111.944420   
              0                        560               100.469704   
2             0                          1               422.815399   
...                                    ...                      ...   
97            0                          2               128.007904   
98            1                          2               282.845428   
              0                          3               294.921021   
99            1                          2               455.960449   
              0                        562               462.556213   

                               raw_em_energy_supercls_sum  \
eventInternal caloparticle_id                               
0             1                                278.767120   
              0                                276.298523   
1             1                                111.944420   
              0                                100.469704   
2             0                                419.481598   
...                                                   ...   
97            0                                128.007904   
98            1                                282.587219   
              0                                294.648163   
99            1                                455.477936   
              0                                462.556213   

                               regressed_energy_supercls_sum  \
eventInternal caloparticle_id                                  
0             1                                   296.012756   
              0                                   296.585327   
1             1                                   117.887543   
              0                                   109.036896   
2             0                                   470.273895   
...                                                      ...   
97            0                                   139.016571   
98            1                                   308.233307   
              0                                   314.628693   
99            1                                   487.438324   
              0                                   484.305634   

                               regressed_energy_CP  raw_energy_CP  \
eventInternal caloparticle_id                                       
0             1                         289.701660     294.612671   
              0                         289.701660     287.145752   
1             1                         117.821510     114.581795   
              0                         117.821510     112.787537   
2             0                         425.282135     430.134155   
...                                            ...            ...   
97            0                         140.290359     141.977020   
98            1                         312.740784     319.028168   
              0                         312.740784     314.584290   
99            1                         465.985840     468.287537   
              0                         465.985840     469.730469   

                               barycenter_x_CP  barycenter_y_CP  \
eventInternal caloparticle_id                                     
0             1                      97.343002        60.377537   
              0                     -96.854935       -62.194397   
1             1                      70.818954        35.481270   
              0                     -69.216713       -37.312813   
2             0                     -38.978916        62.535175   
...                                        ...              ...   
97            0                     112.020416